In [1]:
import sys
sys.path.append('..')

# Import Libraries
from Utils.Accuracy_measures import topk_accuracy
from Utils.TinyImageNet_loader import get_tinyimagenet_dataloaders
from Utils.Num_parameter import count_parameters
from Models.vit_base import VisionTransformer as vit_base
from Models.vit_tensorized import VisionTransformer as vit_tensorized


import torchvision.transforms as transforms
from torch import nn
from torch import optim

import time
import torch
import os

from torchvision.models import vit_b_16

/home/bady/Desktop/TAVERN/Tests/../Utils/TinyImageNet_loader.py:105: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if train_size is not  'default':


ModuleNotFoundError: No module named 'Models.vit_base'

In [4]:
 # Set up the transforms and train/test loaders
image_size = 64
import torchvision.transforms as transforms


tiny_transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize((image_size, image_size)), 
        transforms.RandomCrop(image_size, padding=5),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
tiny_transform_val = transforms.Compose([
        transforms.Resize((image_size, image_size)), 
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
tiny_transform_test = transforms.Compose([
        transforms.Resize((image_size, image_size)), 
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])


train_loader, test_loader, _ = get_tinyimagenet_dataloaders(
                                                        data_dir = '../datasets',
                                                        transform_train=tiny_transform_train,
                                                        transform_val=tiny_transform_val,
                                                        transform_test=tiny_transform_test,
                                                        batch_size=64,
                                                        image_size=image_size, 
                                                        train_size= 70000)

In [2]:
for i in [4,16,32,64]:
    device = 'cuda'
    batch_size = i
    dummy = torch.rand(batch_size, 3, 224, 224).to(device)
    print(f'Current shape is : {dummy.shape}')

    model1 = vit_b_16()
    model1.heads = nn.Sequential(
        nn.Linear(768,200)
    )
    model1 = model1.to(device)

    model2 = vit_tensorized(input_size=(batch_size,3,224,224),
                patch_size=16,
                num_classes=200,
                embed_dim=(16,16,3),
                num_heads=(2,2,3),
                num_layers=12,
                mlp_dim=(32,32,3),
                dropout=0.1,
                bias=True,
                out_embed=True,
                device=device,
                ignore_modes=(0,1,2),
                Tensorized_mlp=True).to(device)

    model3 = vit_base(input_size=(batch_size,3,224,224),
                patch_size=16,
                num_classes=200,
                embed_dim=16*16*3,
                num_heads=2*2*3,
                num_layers=12,
                mlp_dim=32*32*3,
                dropout=0.1,
                bias=True,
                out_embed=True,
                device=device,
                ignore_modes=None,
                Tensorized_mlp=False).to(device)
    
    st = time.time()
    model1(dummy)
    elapsed1 =  time.time() - st

    st = time.time()
    model2(dummy)
    elapsed2 =  time.time() - st

    st = time.time()
    model3(dummy)
    elapsed3 =  time.time() - st

    print(f'{batch_size}, {elapsed1}, {elapsed3}, {elapsed2}')

Current shape is : torch.Size([4, 3, 224, 224])
4, 0.053879737854003906, 0.003330230712890625, 0.026360750198364258
Current shape is : torch.Size([16, 3, 224, 224])
16, 0.005127906799316406, 0.003444671630859375, 0.029513835906982422
Current shape is : torch.Size([32, 3, 224, 224])
32, 0.0051348209381103516, 0.003399372100830078, 0.05140089988708496
Current shape is : torch.Size([64, 3, 224, 224])
64, 0.00629115104675293, 0.0035674571990966797, 0.23940634727478027
